In [1]:
import numpy as np
import pandas as pd
import ssl

#  Code for deriving numbers of recipes /vegan/ veggi/ average calories/ percentage values 

In [460]:
recipes= pd.read_csv("recipes_dropnancuisines_ratings_veggie.csv")

In [461]:
#number of recipes per country
c= recipes["country"].value_counts()
df1=pd.DataFrame(c)
df1.columns = ['n_recipes']
df1['n_recipes']=df1['n_recipes'].astype(float)
df1.reset_index(inplace=True)
df1.columns = ['country','n_recipes']

#seperate veggi and vegan and count recipes per country
df2= recipes.loc[recipes['vegetarian_vegan'] == 'vegan'].groupby("country")['vegetarian_vegan'].value_counts()
df3= recipes.loc[recipes['vegetarian_vegan'] == 'vegetarian'].groupby("country")['vegetarian_vegan'].value_counts()
vegan= pd.DataFrame(df2)
vegan.columns = ['vegan_n_recipes']
vegan['vegan_n_recipes']=vegan['vegan_n_recipes'].astype(float)
vegan.reset_index(inplace=True)
vegan.drop(labels=['vegetarian_vegan'], axis=1, inplace=True)

vegetarian= pd.DataFrame(df3)
vegetarian.columns = ['veggi_n_recipes']
vegetarian['veggi_n_recipes']= vegetarian['veggi_n_recipes'].astype(float)
vegetarian.reset_index(inplace=True)
vegetarian.drop(labels=['vegetarian_vegan'], axis=1, inplace=True)

# get first element of nutrition cell values
df6= recipes[['country', 'nutrition']]
df6['firstcut']= df6['nutrition'].str.split(',').str[0]
df6['calories']= df6['firstcut'].str.split("[").str[1]
df6= df6[['country', 'calories']]
df6['calories']= df6['calories'].astype(float)
# calculate average calories per country
df4= df6.groupby('country')['calories'].sum()/recipes["country"].value_counts()
average_calories= pd.DataFrame(df4)
average_calories.columns = ['average_calories']
average_calories.reset_index(inplace=True)
average_calories.columns = ['country','average_calories']
average_calories['average_calories']= average_calories['average_calories'].round(2)

#merge all df's 
step1= pd.merge(df1, vegetarian, on= "country", how='outer')
step2 = pd.merge(step1, vegan, on= "country", how='outer')
final= pd.merge(step2, average_calories, on= "country", how='outer')

<ipython-input-461-65b9dbd59fbf>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['firstcut']= df6['nutrition'].str.split(',').str[0]
<ipython-input-461-65b9dbd59fbf>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['calories']= df6['firstcut'].str.split("[").str[1]


In [462]:
#calculate the percentages
g= final.assign(percentage_veggi_n_recipes = lambda x: (x['veggi_n_recipes'] /x['n_recipes'] * 100).round(2))
p_list= g.assign(percentage_vegan_n_recipes = lambda x: (x['vegan_n_recipes'] /x['n_recipes'] * 100).round(2))

In [463]:
number_res= final.n_recipes.sum()
number_veggi= final.veggi_n_recipes.sum()
number_vegan= final.vegan_n_recipes.sum()
p_veggi= (number_veggi/number_res*100).round(2)
p_vegan= (number_vegan/number_res*100).round(2)

In [464]:
p_list= p_list.fillna(0)

In [465]:
# append world data
p_list=p_list.append({'country':'world','n_recipes':number_res,'veggi_n_recipes':number_veggi, 'vegan_n_recipes':number_vegan, 'average_calories':0,
               'percentage_veggi_n_recipes':p_veggi,'percentage_vegan_n_recipes':p_vegan}, ignore_index=True)

In [466]:
# create coloumn for non defined as veggi or vegan 
p_list["percentage_non_type"]= 100-p_list["percentage_veggi_n_recipes"]-p_list["percentage_vegan_n_recipes"]

In [165]:
#test to filter by country
p_list.loc[p_list['country'] == 'english']

,country,n_recipes,veggi_n_recipes,vegan_n_recipes,average_calories,percentage_veggi_n_recipes,percentage_vegan_n_recipes,percentage_non_type
10,english,1151.0,286.0,47.0,516.95,24.85,4.08,71.07


In [ ]:
#save as csv
p_list.to_csv("cuisine_percentage_number_recipes.csv",index=False)

# Code for recipe search data 

**not needed anymore**

In [30]:
recipes.drop(labels=['Unnamed: 0','n_ingredients','submitted', 'Unnamed: 0.1','tags', 'nutrition','n_steps', 'nutrition','n_steps','contributor_id','steps','description'], axis=1, inplace=True)

,name,id,minutes,ingredients,region,country,vegetarian_vegan,rating
0,arriba baked winter squash mexican style,137739,55,"['winter squash', 'mexican seasoning', 'mixed ...",central-american,mexican,vegetarian,5.0
1,a bit different breakfast pizza,31490,30,"['prepared pizza crust', 'sausage patty', 'egg...",north-american,american,NaN,3.5
2,amish tomato ketchup for canning,44061,190,"['tomato juice', 'apple cider vinegar', 'sugar...",north-american,american,vegetarian,5.0
3,apple a day milk shake,5289,0,"['milk', 'vanilla ice cream', 'frozen apple ju...",north-american,american,NaN,5.0
4,aww marinated olives,25274,15,"['fennel seeds', 'green olives', 'ripe olives'...",north-american,canadian,vegan,2.0


In [31]:
#save as csv or optional as json
#recipes.to_json("data_recipe_search.json",orient="records")
recipes.to_csv("data_recipe_search.csv",index=False)

# Code for deriving top10 ingredient list for each Country

In [680]:
#load dataset
ingredients = pd.read_csv("recipes_dropnancuisines_ratings_veggie.csv")

In [681]:
#coloumns to keep
keep= ["country", "ingredients"]
ingredients = ingredients[keep]
#set coloumn country to index
ingredients = ingredients.set_index('country')
#split by comma
ing= ingredients.apply(lambda x: x.str.split(',').explode())
#delete square brackets
ing = ing.ingredients.str.strip('[]')
#delete  '
ing= ing.str.replace("'", "")
#delete taps
ing = ing.str.strip()

In [682]:
#store as df
ing= pd.DataFrame(ing)
ing =ing.reset_index()
#get counts of ingredients per country
grouped_list= ing.groupby("country")["ingredients"].value_counts()
#store as df
grouped= pd.DataFrame(grouped_list)
#rename columns
grouped.columns = ['ingredient_counts']
grouped=grouped.reset_index()

In [683]:
#store not needed values in lists
not_keep= ['caribbean','micro-melanesia','middle-eastern','scandinavian','scottish','welsh']
no_top_10_ingredient= ['salt','sugar','pepper','brown sugar', 'water']
#kepp the ones that do not contain above values
grouped = grouped[grouped.country.isin(not_keep) == False]
grouped = grouped[grouped.ingredients.isin(no_top_10_ingredient) == False]
#grouped.to_csv("grouped_ingredient_list.csv",index=False)

In [684]:
#store countries in a list
countries= list(grouped.country)
countries =list(set(countries))
grouped_ingredients = pd.DataFrame(columns =["country", "ingredients","ingredient_counts"])


In [685]:
#append first 15 entries to df 
for i in range (len(countries)):
    x= grouped.loc[grouped['country'] == countries[i]].head(10)
    grouped_ingredients= grouped_ingredients.append(x, ignore_index = True)  

In [686]:
# Test sorting by country
#grouped_ingredients.loc[grouped_ingredients['country'] == "american"]

In [687]:
t= grouped_ingredients.groupby('country')
for group in t: 
    indices = list(np.where(grouped_ingredients["ingredients"]=='egg')[0])
    eggs_index= list(np.where(grouped_ingredients["ingredients"]=='eggs')[0])

In [688]:
for k in range (len(eggs_index)):
    for j in range (len(indices)):
        for i in range (len(countries)):
            if((grouped_ingredients.country.iloc[eggs_index[k]]== countries[i])& (grouped_ingredients.country.iloc[indices[j]]== countries[i])):
                r= grouped_ingredients.iat[eggs_index[k],2]+grouped_ingredients.iat[indices[j],2]
                #print('hello',countries[i],r)
                grouped_ingredients.iat[eggs_index[k],2]=r              
d_egg= ["egg"]
grouped_ingredients =  grouped_ingredients[ grouped_ingredients.ingredients.isin(d_egg) == False]

In [689]:
top6_ingredients_countries = pd.DataFrame(columns =["country", "ingredients","ingredient_counts"])
#reduce to top 6 ingredients 
for i in range (len(countries)):
    x=  grouped_ingredients.loc[ grouped_ingredients['country'] == countries[i]].head(6)
    top6_ingredients_countries = top6_ingredients_countries .append(x, ignore_index = True)

In [691]:
#generate World ingredients counts
world_ingredients_list= grouped_ingredients.groupby("ingredients")["ingredient_counts"].sum()
world_ingredients_list= pd.DataFrame(world_ingredients_list)
world_ingredients_list.columns = ['ingredient_counts']
world_ingredients_list= world_ingredients_list.reset_index()
world_ingredients_list= world_ingredients_list.sort_values(ascending=False,by=['ingredient_counts'])
#display top 6
world_ingredients_list= world_ingredients_list.head(6)
world_ingredients_list["country"]= "world"

In [693]:
#append world_ingredients_list to top6 and save as csv 
top6_ingredients= top6_ingredients_countries.append(world_ingredients_list)
top6_ingredients= top6_ingredients.sort_values(by=['country','ingredient_counts'],ascending=False)
top6_ingredients.to_json("top6_ingredients.json",orient="records")